## Explore LDSim GPU data via Jupyter notebook

### This notebook relies on several free and open-source Python packages, such as [Leafmap](https://leafmap.org), [plotly](https://plotly.com/python/getting-started/), [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/), [GeoPandas](https://geopandas.org/en/stable/), [Fiona](https://pypi.org/project/Fiona/), [Rasterio](https://rasterio.readthedocs.io/en/stable/), and [matplotlib](https://matplotlib.org/) that each contribute different tools in Python that empower users to do interactive mapping, geospatial analysis, and data visualization within a Jupyter notebook.

#### For example, Leafmap is a Python package that enables geospatial analysis and interactive mapping within a Jupyter environment. Leafmap leverages several other open-source packages, such as `ipyleaflet`, which enables complex interactive mapping, WhiteboxTools, and `whiteboxgui`, which allows for complex geospatial analysis; and `ipywidgets` for designing interactive graphical user interfaces (GUIs). Leafmap's interactive *mapping* interface is contrasted with plotly, an interactive *plotting* library that supports over 40 unique chart types. 

**<span style="color:red">NOTE: Many of these tools will need ample time to run depending on contributing factors such as the complexity of the data package, network speed, and server load.</span>**

### Begin by installing necesssary packages: [leafmap](https://leafmap.org)  and [GeoPandas](https://geopandas.org/en/stable/).

**NOTE:** If you already have the following packages installed, such as leafmap and geopandas, comment out the code below by adding "#" in front of each line. 
`# !pip install leafmap`

In [ ]:
!pip install leafmap
!pip install geopandas 
!pip install mapclassify 
!pip install topojson # can use to simplify geometry and speed up rendering/processing times
!pip install chart_studio # Adds Plotly dependencies
!pip install rasterio 

### Import the packages:

In [ ]:
import leafmap
from leafmap import leafmap
import leafmap.colormaps as cm
import pandas as pd
import geopandas as gpd
import fiona
import rasterio
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import mapclassify as mpcl
import topojson as tp
from shapely import geometry
import numpy
import json
import plotly.express as px
import pyproj

### Make sure that leafmap package is up-to-date:

In [ ]:
leafmap.update_package()

### Use GeoPandas to read the LDSim **GPU** [FlatGeobuf](https://flatgeobuf.org/) data layer from VPDC HTTPS URL and return it as a [GeoDataFrame](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html).

In [ ]:
yuba_gpu = gpd.read_file(
    "https://storage.googleapis.com/way-find.com/vpdc/data/LDSim/region/Yuba/ldsim_outputs/yuba_gpu.fgb"
)

### Use [TopoJson](https://pypi.org/project/topojson/1.0rc4/) to simplify the yuba's geometry in order to speed up rendering 

**<span style="color:red">WARNING: Only use the simplified geodataframe for visualizing the data. Any spatial analysis, such as determining the area of polygons, etc. should be done with the original "yuba_gpu" above.</span>**

In [ ]:
topo = tp.Topology(yuba_gpu.to_crs({'init':'epsg:3857'}), prequantize=False)
yuba_gpu_smpl = topo.toposimplify(1).to_gdf()

## Review the hosted LDSim GPU data for the Yuba region

### Visualize the GPU data by creating a static plot 

In [ ]:
yuba_gpu_smpl.plot(figsize=(15, 10))

### Examine the Yuba gpu associated attribute data

In [ ]:
yuba_gpu_smpl.head()

### Plot composite FRID (Fire Return Interval Departure) estimates with an accompanying legend

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)

yuba_gpu_smpl.plot(
    column='FRID', 
    ax=ax, 
    cmap='RdYlGn_r', 
    legend=True,
    legend_kwds={'label': "Fire Return Interval Departure (FRID) at the 5m cell level",
                        'orientation': "horizontal"})

ax.set_axis_off();

## Explore the LDSim GPU data deeper by creating and symbolizing an interactive map

### Create a map instance, add, and explore the raw LDSim GPU [GeoDataFrame](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html) data: 

**_NOTE: To explore the GPU data's attributes, hover over a polygon in the map._**

In [ ]:
m = leafmap.Map(center=[39.40, -120.50], zoom=11, height=600, widescreen=False) #create the map instance

m.add_gdf(yuba_gpu_smpl, layer_name="Yuba GPU") #add the layer to the map instance

m #draw the map instance

### Review the GPU file's data dictionary to better understand the attributes of the data: 

In [ ]:
gpu_data_dictionary = "https://storage.googleapis.com/way-find.com/vpdc/data/LDSim/region/Yuba/ldsim_outputs/gpuResults/documents/Yuba_gpuResults_dataDictionary.csv"
gpu_dd = pd.read_csv(gpu_data_dictionary)
gpu_dd

### Save the Yuba GPU data dictionary csv file to your local directory to review in tandem with this notebook: 

**<span style="color:red">TODO: VERIFY THAT THE DATASET IS AVAILABLE TO OTHER USERS.</span>**

In [ ]:
gpu_dd.to_csv('yuba_gpu_dataDictionary.csv')## Save the CSV file to root of local directory

## Symbolize the gpu units to match various criteria

### Symbolize the GPU unit to show the Fire Return Interval Departure (FRID) at the 5m cell level classsified using Natural Breaks. 

In [ ]:
m = leafmap.Map(
    google_map="HYBRID", 
    draw_control=False,
    measure_control=False,
    center=[39.40, -120.50],
    zoom=11,
)

m.add_data(
    yuba_gpu_smpl, column='FRID', scheme='NaturalBreaks', cmap='RdYlGn_r', legend_title='FRID (Fire Return Interval Departure)', layer_name = 'Yuba FRID', k=10
)

m


The map above shows the FRID (Fire Return Interval Departure) classified at the GPU scale. Positive values represent a contemporary Fire Return Interval > the pre-settlement Fire Return Interval and, conversely, negative values represent a contemporary Fire Return Interval < the pre-settlement Fire Return Interval.

### Review data with Plotly graphing package

In [ ]:
yuba_gpu_smpl.to_crs(epsg=4326, inplace=True)

#gdf.set_index('id', inplace=True)
fig = px.choropleth_mapbox(yuba_gpu_smpl,
                           geojson=yuba_gpu_smpl['geometry'],
                           locations=yuba_gpu_smpl.index,
                           color='FRID',
                           color_continuous_scale="rdylgn_r",
                           mapbox_style="carto-positron",
                           center={'lat':39.5597, 'lon':-120.82873},
                           zoom=9,
                           opacity=1,
                           height=900
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()